In [9]:
import os
import copy
import numpy as np
import pandas as pd
import tensorflow as tf
import torchaudio
import torch
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Resizing, Conv2D, Dropout, BatchNormalization, MaxPool2D, Flatten, Dense
from tqdm import tqdm
import tensorflow_addons as tfa

F1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)

# Configuración de parámetros
class Config:
    def __init__(self, architecture='CNN_LF', epochs=50, sample_rate=None, time=5, n_splits=5, window_size=5):
        self.architecture = architecture
        self.epochs = epochs
        self.sample_rate = sample_rate
        self.time = time
        self.n_splits = n_splits
        self.window_size = window_size

# Funciones de Preprocesamiento
def load_audio_data(directory, window_size, sample_rate):
    audio_dict = {}
    for file_name in os.listdir(directory):
        if file_name.endswith(".wav"):
            waveform, sr = torchaudio.load(os.path.join(directory, file_name))
            num_windows = int(waveform.shape[1] / (window_size * sample_rate))
            for i in range(num_windows):
                start = i * window_size * sample_rate
                end = (i + 1) * window_size * sample_rate
                audio_dict[f"{file_name}_{i}"] = waveform[:, start:end].numpy()
    return audio_dict, sr

def preprocess_audio(audio_dict, sample_rate):
    audio_dict = copy.deepcopy(audio_dict)
    n_mels = 128
    n_fft = int(sample_rate * 0.029)
    hop_length = int(sample_rate * 0.010)
    win_length = int(sample_rate * 0.025)

    for filename, waveform in tqdm(audio_dict.items(), desc='MELSPECTROGRAM'):
        waveform = torch.from_numpy(waveform)
        spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=n_fft, n_mels=n_mels, hop_length=hop_length, win_length=win_length)(waveform)
        spec = torchaudio.transforms.AmplitudeToDB()(spec)
        spec = spec.numpy()
        spec = (spec - spec.min()) / (spec.max() - spec.min())
        audio_dict[filename] = spec
    return audio_dict

def pad_and_crop_spectrograms(spectrograms, target_shape=(128, 128)):
    padded_spectrograms = []
    for spec in spectrograms:
        if spec.shape[0] > target_shape[0]:
            spec = spec[:target_shape[0], :]
        if spec.shape[1] > target_shape[1]]:
            spec = spec[:, :target_shape[1]]
        
        pad_width = [(0, max(0, target_shape[0] - spec.shape[0])), 
                     (0, max(0, target_shape[1] - spec.shape[1]))]
        
        padded_spec = np.pad(spec, pad_width, mode='constant')
        padded_spectrograms.append(padded_spec)
    return np.array(padded_spectrograms)

def train_test_split_audio(audio_dict):
    df = pd.read_csv('Dataset.csv', usecols=['Participant_ID', 'PHQ-9 Score'], dtype={1: str})
    df['labels'] = np.zeros([len(df),], dtype=int)
    df.loc[df['PHQ-9 Score'] < 10, 'labels'] = 0
    df.loc[df['PHQ-9 Score'] >= 10, 'labels'] = 1

    labels = df.set_index('Participant_ID').to_dict()['labels']

    X, Y = [], []
    for filename, data in tqdm(audio_dict.items(), 'LABEL'):
        ID = filename[:3]
        if ID in labels:
            dep = 0 if labels[ID] == 0 else 1
            [X.append(x) for x in data]
            [Y.append(dep) for x in data]

    X = pad_and_crop_spectrograms(X)
    Y = np.array(Y)

    X = X[..., np.newaxis]
    return X, Y

# Definición de Modelos
def build_model(config):
    if config.architecture == 'CNN_LF':
        return build_CNN_LF_model()
    elif config.architecture == 'LSTM':
        return build_LSTM_model()
    else:
        raise ValueError("Arquitectura no soportada")

def build_CNN_LF_model(input_shape=(128, 128, 1)):
    model = Sequential()
    model.add(Resizing(128, 128, input_shape=input_shape))
    model.add(Conv2D(30, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Conv2D(15, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Flatten())
    model.add(Dense(units=256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid"))
    return model

def build_LSTM_model():
    pass

# Entrenamiento y Evaluación
def train_and_evaluate_model(model, X_train, Y_train, X_test, Y_test, config):
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=["accuracy", 'Precision', 'Recall', F1])
    model.fit(X_train, Y_train, epochs=config.epochs, validation_data=(X_test, Y_test))
    results = model.evaluate(X_test, Y_test)
    
    # Predicciones para métricas adicionales
    Y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = results[1]
    precision = precision_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred)
    
    return results[0], accuracy, precision, recall, f1

# Ejecución Principal
def main():
    config = Config(architecture='CNN_LF', epochs=50, n_splits=5, window_size=5)
    directory = './SM-27'
    
    audio_dict, sample_rate = load_audio_data(directory, config.window_size, config.sample_rate)
    audio_dict = preprocess_audio(audio_dict, sample_rate)
    X, Y = train_test_split_audio(audio_dict)
    
    kfold = KFold(n_splits=config.n_splits, shuffle=True)

    results = []
    
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        model = build_model(config)
        fold_results = train_and_evaluate_model(model, X_train, Y_train, X_test, Y_test, config)
        results.append(fold_results)
    
    results = np.array(results)
    avg_results = np.mean(results, axis=0)
    
    print(f"Loss: {avg_results[0]}, Accuracy: {avg_results[1]}, Precision: {avg_results[2]}, Recall: {avg_results[3]}, F1-score: {avg_results[4]}")

if __name__ == "__main__":
    main()


LABEL: 100%|██████████| 1674/1674 [00:00<00:00, 558129.17it/s]


Epoch 1/50
42/42 [==============================] - 2s 34ms/step - loss: 0.9725 - accuracy: 0.5146 - precision: 0.4788 - recall: 0.5266 - f1_score: 0.5015 - val_loss: 0.6940 - val_accuracy: 0.4925 - val_precision: 0.4840 - val_recall: 0.7469 - val_f1_score: 0.5874
Epoch 2/50
42/42 [==============================] - 1s 22ms/step - loss: 0.8877 - accuracy: 0.5362 - precision: 0.5000 - recall: 0.4895 - f1_score: 0.4947 - val_loss: 0.6933 - val_accuracy: 0.5284 - val_precision: 0.5067 - val_recall: 0.9321 - val_f1_score: 0.6565
Epoch 3/50
42/42 [==============================] - 1s 21ms/step - loss: 0.8633 - accuracy: 0.5497 - precision: 0.5147 - recall: 0.5072 - f1_score: 0.5109 - val_loss: 0.6950 - val_accuracy: 0.4896 - val_precision: 0.4862 - val_recall: 0.9815 - val_f1_score: 0.6503
Epoch 4/50
42/42 [==============================] - 1s 21ms/step - loss: 0.8475 - accuracy: 0.5676 - precision: 0.5348 - recall: 0.5201 - f1_score: 0.5273 - val_loss: 0.6983 - val_accuracy: 0.4896 - val_pr